In [1]:
import tensorflow as tf

In [2]:
a = tf.random.shuffle(tf.range(5)) # 打乱 0 - 5
print(a)

tf.Tensor([1 0 2 3 4], shape=(5,), dtype=int32)


In [3]:
tf.sort(a, direction='DESCENDING') # 降序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0], dtype=int32)>

In [5]:
tf.argsort(a, direction='DESCENDING') # 序号降序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 0, 1], dtype=int32)>

In [7]:
idx = tf.argsort(a, direction='DESCENDING')

In [8]:
tf.gather(a, idx)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0], dtype=int32)>

In [9]:
a = tf.random.uniform([3, 3], maxval=10, dtype=tf.int32)
print(a)
print(tf.sort(a))
print(tf.sort(a, direction='DESCENDING'))
print(tf.argsort(a))

tf.Tensor(
[[2 8 2]
 [8 4 1]
 [3 3 7]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[2 2 8]
 [1 4 8]
 [3 3 7]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[8 2 2]
 [8 4 1]
 [7 3 3]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[0 2 1]
 [2 1 0]
 [0 1 2]], shape=(3, 3), dtype=int32)


In [10]:
# top_k
res = tf.math.top_k(a, 2) # 返回最大的前两个值
print(res.indices)
print(res.values)

tf.Tensor(
[[1 0]
 [0 1]
 [2 0]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[8 2]
 [8 4]
 [7 3]], shape=(3, 2), dtype=int32)


In [18]:
# top_k 应用
# 预测值
prob = tf.constant([
    [0.1, 0.2, 0.7],
    [0.2, 0.7, 0.1]
])
# 目标序号
target = tf.constant([2, 0])
# 求出 top_3 的序号
k_b = tf.math.top_k(prob, 3).indices
print(k_b)
# 转置
k_b = tf.transpose(k_b)
print(k_b)
target = tf.broadcast_to(target, [3, 2])
print(target)
# 如果取 top_1 准确率为 50% 一行中有一个 True 1/2
# 如果取 top_2 准确率为 100%  
tf.equal(k_b, target)

tf.Tensor(
[[2 1 0]
 [1 0 2]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[2 1]
 [1 0]
 [0 2]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[2 0]
 [2 0]
 [2 0]], shape=(3, 2), dtype=int32)


<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True, False],
       [False,  True],
       [False, False]])>

In [20]:
# 封装上面准确率计算为方法
def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.shape[0]
    pred = tf.math.top_k(output, maxk).indices
    pred = tf.transpose(pred, perm=[1, 0])
    target_ = tf.broadcast_to(target, pred.shape)
    correct = tf.equal(pred, target_)
    res = []
    for k in topk:
        correct_k = tf.cast(tf.reshape(correct[:k], [-1]), dtype=tf.float32)
        correct_k = tf.reduce_sum(correct_k)
        acc = float(correct_k / batch_size)
        res.append(acc)
    return res

In [23]:
output = tf.random.normal([10, 6])
output = tf.math.softmax(output, axis=1)
target = tf.random.uniform([10], maxval=6, dtype=tf.int32)
print('prob:', output.numpy())
pred = tf.argmax(output, axis=1)
print('pred:', pred.numpy())
print('label:', target.numpy())
acc = accuracy(output, target, topk=(1, 2, 3, 4, 5, 6))
print('top-1-6 acc:', acc)

prob: [[0.01120587 0.2967659  0.20603049 0.32164657 0.09519781 0.06915337]
 [0.1548031  0.12356821 0.131262   0.20788772 0.07376944 0.30870956]
 [0.08258385 0.18781161 0.17159073 0.29987347 0.19314817 0.06499222]
 [0.53826743 0.0111547  0.2390883  0.07201643 0.08478447 0.05468873]
 [0.07192136 0.14299741 0.04516456 0.33761546 0.32801872 0.07428244]
 [0.06449894 0.24116758 0.0443985  0.07498345 0.01188659 0.56306493]
 [0.18668595 0.18669048 0.17453559 0.04429607 0.25670323 0.1510887 ]
 [0.32413477 0.13630806 0.03555444 0.23615053 0.20881893 0.05903333]
 [0.16397741 0.08915227 0.12183238 0.09122517 0.31899983 0.21481296]
 [0.07793785 0.04870022 0.2456908  0.22384383 0.18956949 0.21425785]]
pred: [3 5 3 0 3 5 4 0 4 2]
label: [3 5 0 2 4 4 3 1 3 3]
top-1-6 acc: [0.20000000298023224, 0.5, 0.5, 0.6000000238418579, 0.800000011920929, 1.0]
